# Modelo 3: LSTM(Bilstm) + CRF + embedding

Realizado por:
- Jose Luis Hincapie Bucheli (2125340)
- Sebastián Idrobo Avirama (2122637)
- Paul Rodrigo Rojas Guerrero (2127891)

---

# Traer información desde Google Drive

Si te encuentras usando Google Colab, probablemente necesites descomentar los siguientes bloques de código y modificar las rutas para que funcione con el tuyo:

In [1]:
# Borrar esto
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import sys
# import os

# sys.path.append('/content/drive/My Drive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/packages') # Poner Rutas de los paquetes aquí
# sys.path.append('/content/drive/My Drive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/embeddings') # Poner Rutas de los embeddings aquí

# path = '/content/drive/My Drive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/packages'

# Se verifica que todo se encuentre funcionando correctamente
# files = os.listdir(path)
# print(files)

['wrapper.py', 'tf2crf.py', 'utils.py', 'crfta.py', '__pycache__']


---

# Instalación e importación de paquetes

In [3]:
try:
    import seqeval
except ModuleNotFoundError as err:
    !pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 748.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ccb68fa270438731dd63faafd371170a17a26908ab5955843f22c6cc276993bc
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [4]:
!pip install tensorflow-addons==.0.23.0
!pip install tensorflow==2.15.0
!pip install keras==2.15.0
!pip install nltk==3.8.1
!pip install seqeval
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.2 MB/s eta 0:00:00


In [5]:
import sys
import os

# Add the directory to the system path
sys.path.append('/packages')

In [6]:
import tensorflow as tf
#matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from itertools import islice

#from tabulate import tabulate
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report as eskclarep
#from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
#from seqeval.metrics import classification_report as seqclarep
from sklearn.preprocessing import LabelBinarizer
from itertools import chain

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten
from tensorflow.keras.optimizers import Adam, schedules
from crfta import CRF as crf4
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

from IPython.core.display import display, HTML

import datetime, os
import random
import seqeval

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Importación de dataset conll2002

In [8]:
import nltk
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2002.zip.


['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [9]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
eval_sents = list(nltk.corpus.conll2002.iob_sents('esp.testa'))
print(len(train_sents),len(max(train_sents,key=len)))
print(len(test_sents),len(max(test_sents,key=len)))
print(len(eval_sents),len(max(eval_sents,key=len)))

8323 1238
1517 202
1915 141


In [10]:
print(train_sents[0])

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


# Parte 1: Preprocesamiento de los datos

En esta parte, los datos que serán utilizados -conll2002- deben de ser procesados de tal manera que los tokens y labels sean separados y accedidos por medio de listas de índices y convertir las sentencias de representaciones textuales a secuencias de índices.

In [11]:
def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [12]:
X_train = [sent2tokens(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2tokens(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

X_eval = [sent2tokens(s) for s in eval_sents]
y_eval = [sent2labels(s) for s in eval_sents]

In [13]:
print(X_train[2])
print(y_train[2])

['El', 'Abogado', 'General', 'del', 'Estado', ',', 'Daryl', 'Williams', ',', 'subrayó', 'hoy', 'la', 'necesidad', 'de', 'tomar', 'medidas', 'para', 'proteger', 'al', 'sistema', 'judicial', 'australiano', 'frente', 'a', 'una', 'página', 'de', 'internet', 'que', 'imposibilita', 'el', 'cumplimiento', 'de', 'los', 'principios', 'básicos', 'de', 'la', 'Ley', '.']
['O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O']


In [14]:
import numpy as np

words, tagsss = set([]), set([])

for s in (X_train + X_eval + X_test):
    for w in s:
        words.add(w.lower())

for ts in (y_train + y_eval + y_test):
    for t in ts:
        tagsss.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

tag2index = {t: i + 2 for i, t in enumerate(list(tagsss))}
tag2index['-PAD-'] = 0  # The special value used to padding
tag2index['-OOV-'] = 1  # The special value used to padding

print (len(word2index))
print (len(tag2index))
print(tag2index)


print(tagsss)

28384
11
{'B-LOC': 2, 'B-ORG': 3, 'B-PER': 4, 'O': 5, 'B-MISC': 6, 'I-LOC': 7, 'I-MISC': 8, 'I-PER': 9, 'I-ORG': 10, '-PAD-': 0, '-OOV-': 1}
{'B-LOC', 'B-ORG', 'B-PER', 'O', 'B-MISC', 'I-LOC', 'I-MISC', 'I-PER', 'I-ORG'}


In [15]:
train_sentences_X, eval_sentences_X, test_sentences_X, train_tags_y, eval_tags_y, test_tags_y = [], [], [], [], [], []

for s in X_train:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in X_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    eval_sentences_X.append(s_int)

for s in X_test:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in y_train:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    train_tags_y.append(s_int)

for s in y_eval:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    eval_tags_y.append(s_int)

for s in y_test:
    s_int = []
    for w in s:
        try:
            s_int.append(tag2index[w])
        except KeyError:
            s_int.append(tag2index['-OOV-'])

    test_tags_y.append(s_int)


In [16]:
print("Longitudes de las Matrices:")
print(len(train_sentences_X))
print(len(eval_sentences_X))
print(len( test_sentences_X))
print(len(train_tags_y))
print(len(eval_tags_y))
print(len(test_tags_y))

print("\nMuestra de Datos presentes en las Matrices con las transformaciones:\n")


print(train_sentences_X[0])
print(eval_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(eval_tags_y[0])
print(test_tags_y[0])


Longitudes de las Matrices:
8323
1915
1517
8323
1915
1517

Muestra de Datos presentes en las Matrices con las transformaciones:

[396, 4371, 8128, 12910, 6567, 18820, 15547, 4371, 3335, 12910, 16715]
[20188, 12857, 4371, 20678, 12910, 6567, 1298, 15547, 4371, 17074, 12910, 16715]
[6190, 7063, 6567, 1298, 15547, 4371, 17074, 12910, 16715]
[2, 5, 2, 5, 5, 5, 5, 5, 3, 5, 5]
[2, 7, 5, 2, 5, 5, 5, 5, 5, 3, 5, 5]
[2, 7, 5, 5, 5, 5, 3, 5, 5]


Una vez con estas variables, se debe de normalizar las matrices a partir de MAX_LENGTH, requiriendo de un array estático para servir como input de la RNN.

In [17]:

MAX_LENGTH=202
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
eval_sentences_X = pad_sequences(eval_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
eval_tags_y = pad_sequences(eval_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

print(train_sentences_X[0])
print(train_sentences_X.shape)
print(eval_sentences_X[0])
print(eval_sentences_X.shape)
print(test_sentences_X[0])
print(test_sentences_X.shape)
print(train_tags_y[0])
print(train_tags_y.shape)
print(eval_tags_y[0])
print(eval_tags_y.shape)
print(test_tags_y[0])
print(test_tags_y.shape)



[  396  4371  8128 12910  6567 18820 15547  4371  3335 12910 16715     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

Y una vez normalizadas las matrices, se procede a representar las listas de indices de etiquetas a una representación one-hot para el procesamiento con el RNN

In [18]:
def to_categoricals(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [19]:
def encode(data):
    print('Shape of data (BEFORE encode): %s' % str(data.shape))
    encoded = to_categorical(data)
    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
    return encoded

In [20]:
cat_train_tags_y = to_categoricals(train_tags_y, len(tag2index))
cat_eval_tags_y  = to_categoricals(eval_tags_y, len(tag2index))
cat_test_tags_y  = to_categoricals(test_tags_y, len(tag2index))

print(cat_train_tags_y[1])
print(len(cat_train_tags_y))
print(cat_train_tags_y.shape)
print(len(cat_test_tags_y))

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
8323
(8323, 202, 11)
1517


# Parte 2: Entrenamiento del modelo de red

Se define la red neuronal para el trabajo de NER usando el modelo LSTM(Bilstm) + CRF + embedding, importando los módulos necesarios para el mismo y también parametrizando la RNN a entrenar.
- Se trae el embedding con tamaño de 300.
- Se genera el modelo LSTM(Bilstm) + CRF
- Se genera una capa de masking para que ignore los valores de 0

Una vez hecho esto, se procede a entrenar la red neuronal y a testear los resultados de la misma

In [26]:
from tensorflow.keras.layers import Embedding
from gensim.models import KeyedVectors
import numpy as np

word2vec_model = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Ingeniería de sistemas/Semestre 7/PLN/Taller 3 Personal/embeddings/word2vec_jose_sebastian_paul300.txt",
                                                   binary=False)

word_vectors = word2vec_model
vocab = word2vec_model.index_to_key

# Aquí se inicializa el embedding
embedding_matrix = np.zeros((len(vocab), word_vectors.vector_size))

# Se llena el embedding con los datos obtenidos de Word2Vec para ser usado
for i, word in enumerate(vocab):
    embedding_matrix[i] = word_vectors[word]


In [32]:
# TO-DO: No se ha hecho nada diferente por el momento

from tf2crf import CRF as crf6
from wrapper import ModelWithCRFLoss, ModelWithCRFLossDSCLoss
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
from tensorflow.keras.layers import Concatenate, Lambda, Input, LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, InputLayer, Activation, Flatten, Masking
from tensorflow.keras.optimizers import Adam, schedules
input = Input(shape=(MAX_LENGTH,))
word_embedding_size = 300
EMBED_DIM=300
# Embedding Layer
model = Embedding(input_dim=len(vocab),
                             output_dim=word_embedding_size,
                             weights=[embedding_matrix],
                             mask_zero=False)(input)

# BI-LSTM Layer
model = Bidirectional(LSTM(units=50,
                     return_sequences=True,
                     dropout=0.5,
                     recurrent_dropout=0.5))(model)
model  = Dropout(0.5, name='dropout_lstm')(model)
model  = Dense(units=EMBED_DIM * 2, activation='relu')(model)
model  = Dense(units=len(tag2index), activation='relu')(model)

model  = Masking(mask_value=0.,input_shape=(MAX_LENGTH, len(tag2index)))(model)


crf = crf6(units=len(tag2index), name="ner_crf")
predictions = crf(model)

base_model = Model(inputs=input, outputs=predictions)
model = ModelWithCRFLoss(base_model, sparse_target=True)

model.compile(optimizer='adam')
#model.summary()

In [33]:
history= model.fit(train_sentences_X, cat_train_tags_y,
                       validation_data=(eval_sentences_X, cat_eval_tags_y),
                       batch_size=128,
                       epochs=50,
                       verbose=2)

Epoch 1/50
66/66 - 70s - loss: 68.7013 - accuracy: 0.9103 - val_loss_val: 20.1166 - val_val_accuracy: 0.9804 - 70s/epoch - 1s/step
Epoch 2/50
66/66 - 59s - loss: 18.5656 - accuracy: 0.9806 - val_loss_val: 16.9033 - val_val_accuracy: 0.9805 - 59s/epoch - 890ms/step
Epoch 3/50
66/66 - 59s - loss: 15.6048 - accuracy: 0.9807 - val_loss_val: 14.9866 - val_val_accuracy: 0.9807 - 59s/epoch - 891ms/step
Epoch 4/50
66/66 - 59s - loss: 13.5064 - accuracy: 0.9812 - val_loss_val: 13.6933 - val_val_accuracy: 0.9813 - 59s/epoch - 888ms/step
Epoch 5/50
66/66 - 59s - loss: 12.0751 - accuracy: 0.9822 - val_loss_val: 12.8275 - val_val_accuracy: 0.9819 - 59s/epoch - 891ms/step
Epoch 6/50
66/66 - 59s - loss: 10.8910 - accuracy: 0.9834 - val_loss_val: 12.0382 - val_val_accuracy: 0.9828 - 59s/epoch - 893ms/step
Epoch 7/50
66/66 - 59s - loss: 9.7544 - accuracy: 0.9850 - val_loss_val: 11.4529 - val_val_accuracy: 0.9837 - 59s/epoch - 895ms/step
Epoch 8/50
66/66 - 59s - loss: 8.8540 - accuracy: 0.9863 - val_los

In [34]:
print(tag2index)
print(test_sentences_X)
y_pred= model.predict(test_sentences_X)
print(y_pred.shape)
print(y_pred)

{'B-LOC': 2, 'B-ORG': 3, 'B-PER': 4, 'O': 5, 'B-MISC': 6, 'I-LOC': 7, 'I-MISC': 8, 'I-PER': 9, 'I-ORG': 10, '-PAD-': 0, '-OOV-': 1}
[[ 6190  7063  6567 ...     0     0     0]
 [28351     0     0 ...     0     0     0]
 [16668 26624 13873 ...     0     0     0]
 ...
 [ 6190 17225 13507 ...     0     0     0]
 [ 3603 11086 24629 ...     0     0     0]
 [ 6190 23086 19746 ...     0     0     0]]
48/48 [==============================] - 4s 59ms/step
(1517, 202)
[[2 7 5 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 ...
 [5 2 5 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 [5 5 5 ... 0 0 0]]


In [35]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_pred = logits_to_tokens(y_pred, index2tag)
print(y1_pred[10])

{2: 'B-LOC', 3: 'B-ORG', 4: 'B-PER', 5: 'O', 6: 'B-MISC', 7: 'I-LOC', 8: 'I-MISC', 9: 'I-PER', 10: 'I-ORG', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PA

In [37]:
#print(Y_test[4])
print(test_tags_y.shape)

(1517, 202)


In [38]:
from utils import build_matrix_embeddings as bme, plot_model_performance, logits_to_tokens, report_to_df
index2tag = {i: t for t, i in tag2index.items()}
print(index2tag)
y1_true = logits_to_tokens(test_tags_y, index2tag)
print(y1_true[10])

{2: 'B-LOC', 3: 'B-ORG', 4: 'B-PER', 5: 'O', 6: 'B-MISC', 7: 'I-LOC', 8: 'I-MISC', 9: 'I-PER', 10: 'I-ORG', 0: '-PAD-', 1: '-OOV-'}
['O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-'

## Calidad obtenida del modelo

Se evalua la calidad de los resultados del modelo usando las siguientes métricas: *precision, recall, accuracy* y *F1-score*.

In [39]:
from seqeval.metrics import classification_report as seqclarep
from seqeval.metrics import precision_score, recall_score, f1_score, accuracy_score
print("precision: {:.1%}".format(precision_score(y1_true, y1_pred)))
print("   recall: {:.1%}".format(recall_score(y1_true,    y1_pred)))
print(" accuracy: {:.1%}".format(accuracy_score(y1_true,  y1_pred)))
print(" F1-score: {:.1%}".format(f1_score(y1_true,        y1_pred)))

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -PAD- seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


precision: 70.6%
   recall: 64.8%
 accuracy: 99.1%
 F1-score: 67.5%


Se genera un reporte de la clasificación basada en las etiquetas predichas y las reales, esto brindado en un DataFrame.

In [40]:
import pandas as pd
li1 = sum(y1_true, [])
li2 = sum(y1_pred, [])

results = pd.DataFrame(columns=['Expected', 'Predicted'])

results['Expected'] = li1
results['Predicted'] = li2

In [41]:
from sklearn.metrics import classification_report as eskclarep
report = eskclarep(results['Expected'], results['Predicted'])

print(report_to_df(report))

  Class Name precision recall f1-score support
0      -PAD-      1.00   1.00     1.00  254901
1      B-LOC      0.79   0.70     0.74    1084
2     B-MISC      0.46   0.39     0.42     339
3      B-ORG      0.78   0.74     0.76    1400
4      B-PER      0.81   0.72     0.76     735
5      I-LOC      0.74   0.56     0.63     325
6     I-MISC      0.49   0.38     0.43     557
7      I-ORG      0.77   0.62     0.68    1104
8      I-PER      0.90   0.83     0.87     634
9          O      0.97   0.99     0.98   45355


Por último, se realizan evaluaciones del modelo a partir de algunas oraciones.

In [42]:
test_sample1 = [
    "James Rodriguez es el jugador colombiano más importante con Radamel Falcao.".split(),
    " Jugadores de la selección Colombia que juegan en el Reino Unido".split()
]

print(test_sample1)

test_sample2 = [
    "La federación Nacional de cafeteros de Colombia es dirigida por Horacio Sánchez".split(),
    " y se ubica en las ciudades de  Cali y Medellín con el instituto colombiano del café ".split()
]

print(test_sample2)

[['James', 'Rodriguez', 'es', 'el', 'jugador', 'colombiano', 'más', 'importante', 'con', 'Radamel', 'Falcao.'], ['Jugadores', 'de', 'la', 'selección', 'Colombia', 'que', 'juegan', 'en', 'el', 'Reino', 'Unido']]
[['La', 'federación', 'Nacional', 'de', 'cafeteros', 'de', 'Colombia', 'es', 'dirigida', 'por', 'Horacio', 'Sánchez'], ['y', 'se', 'ubica', 'en', 'las', 'ciudades', 'de', 'Cali', 'y', 'Medellín', 'con', 'el', 'instituto', 'colombiano', 'del', 'café']]


In [43]:
test_samples_X = []
for s in test_sample2:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)

test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)
print(test_samples_X.shape)

[[ 6190 13622 28254  9558 13289  9558  3012 24994 25662 25286     1 22896
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [44]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

1/1 [==============================] - 0s 67ms/step
[[ 5  3 10 10 10 10 10  5  5  5  0  9  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 5  5  5  5  5  5  5  5  5  5  5  5  3  5  5  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  

In [45]:
#print(len(predictions))
log_tokens = logits_to_tokens(predictions, {i: t for t, i in tag2index.items()})
print(log_tokens)

[['O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', '-PAD-', 'I-PER', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-

In [46]:
!pip install tabulate
from tabulate import tabulate

heads1 = test_sample2[0]
body1 = [log_tokens[0][:len(test_sample2[0])]]

heads2 = test_sample2[1]
body2 = [log_tokens[1][:len(test_sample2[1])]]

print(tabulate(body1, headers=heads1))

print ("\n")

print(tabulate(body2, headers=heads2))

## postagging Freeling 4.1

## El      hombre   bajo     corre    bajo  el      puente   con  bajo  índice   de  adrenalina  .
## DA0MS0  NCMS000  AQ0MS00  VMIP3S0  SP    DA0MS0  NCMS000  SP   SP    NCMS000  SP  NCFS000     Fp


## pos tagger Stanford NLP

## El      hombre   bajo     corre    bajo  el      puente   con    bajo   índice  de    adrenalina  .
## da0000  nc0s000  aq0000   vmip000  sp000 da0000  nc0s000  sp000  aq0000 nc0s000 sp000 nc0s000     fp

La    federación    Nacional    de     cafeteros    de     Colombia    es    dirigida    por    Horacio    Sánchez
----  ------------  ----------  -----  -----------  -----  ----------  ----  ----------  -----  ---------  ---------
O     B-ORG         I-ORG       I-ORG  I-ORG        I-ORG  I-ORG       O     O           O      -PAD-      I-PER


y    se    ubica    en    las    ciudades    de    Cali    y    Medellín    con    el    instituto    colombiano    del    café
---  ----  -------  ----  -----  ----------  ----  ------  ---  ----------  -----  ----  -----------  ------------  -----  ------
O    O     O        O     O      O           O     O       O    O           O      O     B-ORG        O             O      -PAD-
